In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "3"  # Set the GPUs to use
import torch
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Current cuda device: 0
Count of using GPUs: 1


In [2]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
)

import torch, copy, os
from module.score import evaluation_f1
from module.load_json import *
from module.maps import *
from module.args import print_torch_info
from module.inference import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print_torch_info()

torch.__version__: 1.7.1
torch.cuda.is_available(): True
NGPU: 1


# Paths and Modes

In [3]:
EVAL_MODE = False

RESULT_ID = 'uncleaned_v22_run_1_ensemble_v3'

ACD_1_CHECKPOINT = 'training_results/uncleaned_v22_run_1/acd_klue_roberta_base_mlm_fine_tuned_uncleaned_v22_run_1/checkpoint-37500'
ACD_2_CHECKPOINT = 'training_results/uncleaned_v22_run_1/acd_monologg_koelectra_base_v3_discriminator_uncleaned_v22_run_1/checkpoint-37500'
ACD_3_CHECKPOINT = 'training_results/uncleaned_v22_run_1/acd_snunlp_kr_electra_discriminator_uncleaned_v22_run_1/checkpoint-37500'

ASC_1_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_klue_roberta_base_mlm_fine_tuned_uncleaned_v22_run_1/checkpoint-1608'
ASC_2_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_monologg_koelectra_base_v3_discriminator_uncleaned_v22_run_1/checkpoint-1608'
ASC_3_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_snunlp_kr_electra_discriminator_uncleaned_v22_run_1/checkpoint-1608'

TEST_DATA_PATH = 'dataset/nikluge-sa-2022-test.jsonl'
EVAL_DATA_PATH = 'dataset/nikluge-sa-2022-dev.jsonl'

In [4]:
if EVAL_MODE == True:
    TEST_DATA_PATH = EVAL_DATA_PATH
test_data = jsonlload(TEST_DATA_PATH)
print('>>>>> >>>>> >>>>> ', TEST_DATA_PATH, ' <<<<< <<<<< <<<<<', '\n', sep='')

>>>>> >>>>> >>>>> dataset/nikluge-sa-2022-test.jsonl <<<<< <<<<< <<<<<



# Load Models and Tokenizers

In [5]:
acd_model_1 = AutoModelForSequenceClassification.from_pretrained(ACD_1_CHECKPOINT)
acd_model_2 = AutoModelForSequenceClassification.from_pretrained(ACD_2_CHECKPOINT)
acd_model_3 = AutoModelForSequenceClassification.from_pretrained(ACD_3_CHECKPOINT)
acd_tokenizer_1 = AutoTokenizer.from_pretrained(ACD_1_CHECKPOINT)
acd_tokenizer_2 = AutoTokenizer.from_pretrained(ACD_2_CHECKPOINT)
acd_tokenizer_3 = AutoTokenizer.from_pretrained(ACD_3_CHECKPOINT)

asc_model_1 = AutoModelForSequenceClassification.from_pretrained(ASC_1_CHECKPOINT)
asc_model_2 = AutoModelForSequenceClassification.from_pretrained(ASC_2_CHECKPOINT)
asc_model_3 = AutoModelForSequenceClassification.from_pretrained(ASC_3_CHECKPOINT)
asc_tokenizer_1 = AutoTokenizer.from_pretrained(ASC_1_CHECKPOINT)
asc_tokenizer_2 = AutoTokenizer.from_pretrained(ASC_2_CHECKPOINT)
asc_tokenizer_3 = AutoTokenizer.from_pretrained(ASC_3_CHECKPOINT)

In [6]:
# # v1

# def acd_ensemble(form, acd_pair):
#     acd_model_1.to(device)
#     acd_model_2.to(device)
#     acd_model_3.to(device)
#     acd_model_1.eval()
#     acd_model_2.eval()
#     acd_model_3.eval()

#     acd_encoded_1 = acd_tokenizer_1(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_1 = {k:v.to(device) for k,v in acd_encoded_1.items()}

#     acd_encoded_2 = acd_tokenizer_2(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_2 = {k:v.to(device) for k,v in acd_encoded_2.items()}

#     acd_encoded_3 = acd_tokenizer_3(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_3 = {k:v.to(device) for k,v in acd_encoded_3.items()}

#     with torch.no_grad():
#         acd_outputs_1 = acd_model_1(**acd_encoded_1)['logits']
#         acd_outputs_2 = acd_model_2(**acd_encoded_2)['logits']
#         acd_outputs_3 = acd_model_3(**acd_encoded_3)['logits']

#     acd_outputs = acd_outputs_1 + acd_outputs_2 + acd_outputs_3
#     acd_predictions = acd_outputs.argmax(-1)[0]
    
#     return acd_predictions

In [7]:
# v2

def acd_ensemble(form, acd_pair):
    acd_model_1.to(device)
    acd_model_2.to(device)
    acd_model_3.to(device)
    acd_model_1.eval()
    acd_model_2.eval()
    acd_model_3.eval()

    acd_encoded_1 = acd_tokenizer_1(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_1 = {k:v.to(device) for k,v in acd_encoded_1.items()}

    acd_encoded_2 = acd_tokenizer_2(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_2 = {k:v.to(device) for k,v in acd_encoded_2.items()}

    acd_encoded_3 = acd_tokenizer_3(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_3 = {k:v.to(device) for k,v in acd_encoded_3.items()}

    with torch.no_grad():
        acd_outputs_1 = acd_model_1(**acd_encoded_1)['logits']
        acd_outputs_2 = acd_model_2(**acd_encoded_2)['logits']
        acd_outputs_3 = acd_model_3(**acd_encoded_3)['logits']

    acd_outputs = acd_outputs_1 + acd_outputs_2 + acd_outputs_3
    
    acd_predictions_1 = acd_outputs_1.argmax(-1).squeeze()
    acd_predictions_2 = acd_outputs_2.argmax(-1).squeeze()
    acd_predictions_3 = acd_outputs_3.argmax(-1).squeeze()
    
    if 0 in [acd_predictions_1, acd_predictions_2, acd_predictions_3]:
        return 0
    
    return 1

In [8]:
def asc_ensemble(form, asc_pair):
    asc_model_1.to(device)
    asc_model_2.to(device)
    asc_model_3.to(device)
    asc_model_1.eval()
    asc_model_2.eval()
    asc_model_3.eval()

    asc_encoded_1 = asc_tokenizer_1(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_1 = {k:v.to(device) for k,v in asc_encoded_1.items()}

    asc_encoded_2 = asc_tokenizer_2(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_2 = {k:v.to(device) for k,v in asc_encoded_2.items()}

    asc_encoded_3 = asc_tokenizer_3(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_3 = {k:v.to(device) for k,v in asc_encoded_3.items()}

    with torch.no_grad():
        asc_outputs_1 = asc_model_1(**asc_encoded_1)['logits']
        asc_outputs_2 = asc_model_2(**asc_encoded_2)['logits']
        asc_outputs_3 = asc_model_3(**asc_encoded_3)['logits']

    asc_outputs = asc_outputs_1 + asc_outputs_2 + asc_outputs_3
    
    return asc_outputs

# Inference

In [9]:
for entity_property_pair in entity_property_pairs:
    num_of_ep_pairs = len(entity_property_pair)
    RESULT_SAVE_NAME = f'{RESULT_ID}_num_of_ep_pairs_{num_of_ep_pairs}.json'
    
    pred_data = inference_m_ensemble(acd_ensemble, asc_ensemble, copy.deepcopy(test_data), entity_property_pair)
    
    if EVAL_MODE == False:
        save_path = './'

        jsondump(pred_data, os.path.join(save_path, RESULT_SAVE_NAME))
        pred_data = jsonload(os.path.join(save_path, RESULT_SAVE_NAME))
    
    print(RESULT_SAVE_NAME)    
    print(len(test_data), len(pred_data))
    
    if EVAL_MODE == True:
        print('INFERENCE DATA: ', TEST_DATA_PATH)

        print('EVAL_MODE :', EVAL_MODE)

        result = evaluation_f1(test_data, pred_data)
        print(list(result.items())[0])
        print(list(result.items())[1])
    break

['본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질', '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질', '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질', '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질']


 79%|███████▉  | 1685/2127 [44:47<11:22,  1.55s/it]

패키지/구성품#가격 found.
corrected as 패키지/ 구성품#가격


 81%|████████  | 1721/2127 [45:45<10:39,  1.57s/it]

패키지/구성품#가격 found.
corrected as 패키지/ 구성품#가격


100%|██████████| 2127/2127 [56:35<00:00,  1.60s/it]

number of failed: 102
일반 에어퍼프 느낌으로 생겼지만 발라보면 느낌이 달라서 깜짝 놀랐음

오늘은 또 신기한 패키지의 화장품 더마퓨어클리닉 비타민 >.< ♥

립팔레트를 연상시키는 심볼마크디자인이 상당히 매력적인데요~​​​ ​

다른 메이크업 필요없이 입술에 포인트만 주어도 너무 예쁜 오카 립유어!​

MLBB 립 중에서도 살짝 누디한 계열의 오렌지코랄이라 데일리로 쓰기는 최고네요.

다른 코스메디칼 브랜드에서는 온라인에만 명시하거나 2~3줄로 어느단계에 쓰면 되는지 적어두는데 이렇게 효능이나 함께 쓰면 좋은제품, 특징, 사용법등을 제대로 알려주는 종이가 함께 동봉되어있어서 진짜 신기하더라구요,

관리센터에서 사용하니 말 다했죠 뭐.

좋은데 ㅠ3ㅠ한가지 아쉬운점이 있다면 ~이렇게 겉으로는컬러확인이 불가능하다는 점 ....

라마레따 가을 매트립이 출시된매트 립홀드는넘나 좋게도 ! 저같은 입술 각질녀들이안심하고 사용할 수 있도록립틴트와 함께오일 탑코트가 듀얼로 되어있는 제품이에요

쨍한 느낌의 톤다운 레드로얼굴일 굉장히 화사하게 만들어주더라구요 ~

브러쉬로 정교하게 바른듯디테일하게 립표현을 할 수 있어요

스킨, 로션이 하나로 !

그래서 그건 거의 겨울에나 사용하고 여름에는 안썼는데, 이 베네피트 아이라이너는 토니모리젤아이라이너보다 더 많이 번지더라구요....

하나의 제품으로 샴푸까지 할 수 있으니 좋더라고요.

몽디에스 바쓰앤샴푸머리부터 발끝까지 한 제품으로 깨끗하게 세정할 수 있으니 좋아요.

고도의 과학기술로 앙부땅 리얼 페로몬에 진자 페로몬을 담았다고 하니까 남성적인 매력이 더 어필 되는건가요?!ㅎㅎ

보통 화장품들은 정제수가 대부분인데, CL4 리바이탈라이징C 앰플 슬리핑팩은 물대신 유기농 레몬수를 이용했다고해서 더욱 마음에 들었답니다~

발색쩌는 스타일71 쥬얼리 루즈크림 립스틱 (영상후기 ON)

밀키한 타입 자체가 매우 기분 좋은데요 -

처음 알게 된 브랜드라 어떨지 몰라서 &name& 상처 진정용으로 큐어순 베이비밤만 